# LGBM Model
* RNN was far too slow and hence infeasible given the time limit I am constrained within
* So going to use an LGBM

# Misc / setting up

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
import datetime
from IPython.display import display
from pprint import pprint
import lightgbm as lgbm
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn import preprocessing

In [12]:
import random
SEED = 137
random.seed(SEED)

In [13]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

# loading the data

In [14]:
df_train = pd.read_pickle("../Data/Prepared_Data/supervised/consumption_all.pkl")
display(df_train)
df_preds = pd.read_pickle("../Data/Prepared_Data/supervised/consumption_all_preds.pkl")
display(df_preds)

,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,...,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0x240e5e22734a44a174b7dabcf1ea00d70d9ec168,0,0,0,0,0,0,0,0,0,...,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,0,0,0,0,0,0,0,0,0,...,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,0,0,0,0,0,0,0,0,0,...,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
3,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,0,0,0,0,0,0,0,0,0,...,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,0,0,0,0,0,0,0,0,0,...,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617640,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.0,0.0,21.999000,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500
617641,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.0,0.0,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000
617642,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.0,0.0,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000,19.204000
617643,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.0,0.0,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000,19.204000,17.397000


,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,...,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185515,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185516,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185517,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1185518,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# formatting the df's

In [15]:
#replacing nans with 0 so we can aggregate up the OOF predictions
df_preds["meter_reading"] = df_preds["meter_reading"].fillna(0) 
#dropping the "energy n-k" columns as they are needed for 3D RNN input not 2D LGBM input
df_preds = df_preds.drop(["energy n-7","energy n-6","energy n-5","energy n-4","energy n-3","energy n-2","energy n-1"], axis=1)
df_train = df_train.drop(["energy n-7","energy n-6","energy n-5","energy n-4","energy n-3","energy n-2","energy n-1"], axis=1)

display(df_train)
display(df_preds)

,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,...,month_cos,is_weekend,energy_cluster,num_bedrooms,dwelling_type,detached,flat,semi_detached,terraced,meter_reading
0,0x240e5e22734a44a174b7dabcf1ea00d70d9ec168,0,0,0,0,0,0,0,0,0,...,0.841254,1,0,3.0,semi_detached_house,0.0,0.0,1.0,0.0,11.881495
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,0,0,0,0,0,0,0,0,0,...,0.841254,1,0,3.0,semi_detached_house,0.0,0.0,1.0,0.0,11.881495
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,0,0,0,0,0,0,0,0,0,...,0.841254,1,0,3.0,semi_detached_house,0.0,0.0,1.0,0.0,11.881495
3,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,0,0,0,0,0,0,0,0,0,...,0.841254,1,0,3.0,semi_detached_house,0.0,0.0,1.0,0.0,11.881495
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,0,0,0,0,0,0,0,0,0,...,0.841254,1,0,3.0,semi_detached_house,0.0,0.0,1.0,0.0,11.881495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617640,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.841254,0,1,3.0,detached_house,1.0,0.0,0.0,0.0,18.758500
617641,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.841254,0,1,3.0,detached_house,1.0,0.0,0.0,0.0,21.011000
617642,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.841254,0,1,3.0,detached_house,1.0,0.0,0.0,0.0,19.204000
617643,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.841254,1,1,3.0,detached_house,1.0,0.0,0.0,0.0,17.397000


,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,...,month_cos,is_weekend,energy_cluster,num_bedrooms,dwelling_type,detached,flat,semi_detached,terraced,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,0
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,0
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,0
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,0
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185515,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.841254,0,1,3.0,detached_house,1.0,0.0,0.0,0.0,0
1185516,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.841254,0,1,3.0,detached_house,1.0,0.0,0.0,0.0,0
1185517,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.841254,1,1,3.0,detached_house,1.0,0.0,0.0,0.0,0
1185518,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,...,0.841254,1,1,3.0,detached_house,1.0,0.0,0.0,0.0,0


# Ordinally encoding the ID's
* One hot / binary encoding can actually worsen performance of DT based algorithms
* and LGBM supports categorical values; so no need to use the binary encoded meter_id which we planned for the RNN
* Hence I will now encode the ID ordinally and use that instead
    * If had enough time to run would experiment with both on grid search but will take too long due to exponential growth

In [16]:
#removing the binary encoded ID's as LGBM supports categorical values hence we can just use ordinal encoded ID's for better performance
binary_encoded_cols=[]
for i in range(13):
    binary_encoded_cols.append(f"meter_id_{i}")
df_train=df_train.drop(binary_encoded_cols,axis=1)
df_preds=df_preds.drop(binary_encoded_cols,axis=1)

In [17]:
#ordinally encoding id's
le = preprocessing.LabelEncoder()
le.fit(df_train["meter_id"])

#ordinally encoding with same encoder so will encode the same
df_train.insert(1, "meter_id_ord", le.transform(df_train["meter_id"]))
df_preds.insert(1, "meter_id_ord", le.transform(df_preds["meter_id"]))

display(df_train)
display(df_preds)

,meter_id,meter_id_ord,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,energy_cluster,num_bedrooms,dwelling_type,detached,flat,semi_detached,terraced,meter_reading
0,0x240e5e22734a44a174b7dabcf1ea00d70d9ec168,425,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,0,3.0,semi_detached_house,0.0,0.0,1.0,0.0,11.881495
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,542,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,0,3.0,semi_detached_house,0.0,0.0,1.0,0.0,11.881495
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,585,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,0,3.0,semi_detached_house,0.0,0.0,1.0,0.0,11.881495
3,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,688,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,0,3.0,semi_detached_house,0.0,0.0,1.0,0.0,11.881495
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,793,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,0,3.0,semi_detached_house,0.0,0.0,1.0,0.0,11.881495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617640,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,1,3.0,detached_house,1.0,0.0,0.0,0.0,18.758500
617641,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,1,3.0,detached_house,1.0,0.0,0.0,0.0,21.011000
617642,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,1,3.0,detached_house,1.0,0.0,0.0,0.0,19.204000
617643,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,1,3.0,detached_house,1.0,0.0,0.0,0.0,17.397000


,meter_id,meter_id_ord,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,energy_cluster,num_bedrooms,dwelling_type,detached,flat,semi_detached,terraced,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2073,2018-01-01,1.726064e-02,0.999851,0,1,1,0.540641,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,0
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2073,2018-01-02,3.451614e-02,0.999404,1,2,1,0.540641,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,0
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2073,2018-01-03,5.176135e-02,0.998659,2,3,1,0.540641,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,0
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2073,2018-01-04,6.899114e-02,0.997617,3,4,1,0.540641,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,0
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2073,2018-01-05,8.620038e-02,0.996278,4,5,1,0.540641,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185515,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2018-12-27,-5.176135e-02,0.998659,3,27,12,0.540641,0.841254,0,1,3.0,detached_house,1.0,0.0,0.0,0.0,0
1185516,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2018-12-28,-3.451614e-02,0.999404,4,28,12,0.540641,0.841254,0,1,3.0,detached_house,1.0,0.0,0.0,0.0,0
1185517,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2018-12-29,-1.726064e-02,0.999851,5,29,12,0.540641,0.841254,1,1,3.0,detached_house,1.0,0.0,0.0,0.0,0
1185518,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2018-12-30,-2.449294e-16,1.000000,6,30,12,0.540641,0.841254,1,1,3.0,detached_house,1.0,0.0,0.0,0.0,0


# Ordinally encoding the dwelling_type
* same reasons for replacing the binary encoded meter_id with ordinal encoding; going to replace one hot encodede dwelling type with ordinal encoding

In [18]:
#removing the binary encoded ID's as LGBM supports categorical values hence we can just use ordinal encoded ID's for better performance
onehot_encoded_cols = ["detached","flat","terraced","semi_detached"]
df_train=df_train.drop(onehot_encoded_cols,axis=1)
df_preds=df_preds.drop(onehot_encoded_cols,axis=1)

In [19]:
#ordinally encoding dwelling_type
le = preprocessing.LabelEncoder()
le.fit(df_train["dwelling_type"])

#ordinally encoding with same encoder so will encode the same
df_train.insert(14, "dwelling_type_ord", le.transform(df_train["dwelling_type"]))
df_preds.insert(14, "dwelling_type_ord", le.transform(df_preds["dwelling_type"]))

display(df_train)
display(df_preds)

,meter_id,meter_id_ord,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,energy_cluster,num_bedrooms,dwelling_type,dwelling_type_ord,meter_reading
0,0x240e5e22734a44a174b7dabcf1ea00d70d9ec168,425,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,0,3.0,semi_detached_house,3,11.881495
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,542,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,0,3.0,semi_detached_house,3,11.881495
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,585,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,0,3.0,semi_detached_house,3,11.881495
3,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,688,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,0,3.0,semi_detached_house,3,11.881495
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,793,2017-12-31,1.726064e-02,0.999851,6,31,12,0.540641,0.841254,1,0,3.0,semi_detached_house,3,11.881495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617640,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2017-12-27,-5.176135e-02,0.998659,2,27,12,0.540641,0.841254,0,1,3.0,detached_house,1,18.758500
617641,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2017-12-28,-3.451614e-02,0.999404,3,28,12,0.540641,0.841254,0,1,3.0,detached_house,1,21.011000
617642,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2017-12-29,-1.726064e-02,0.999851,4,29,12,0.540641,0.841254,0,1,3.0,detached_house,1,19.204000
617643,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2017-12-30,-2.449294e-16,1.000000,5,30,12,0.540641,0.841254,1,1,3.0,detached_house,1,17.397000


,meter_id,meter_id_ord,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_ord,month_sin,month_cos,is_weekend,energy_cluster,num_bedrooms,dwelling_type,dwelling_type_ord,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2073,2018-01-01,1.726064e-02,0.999851,0,1,1,0.540641,0.841254,0,0,2.0,terraced_house,4,0
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2073,2018-01-02,3.451614e-02,0.999404,1,2,1,0.540641,0.841254,0,0,2.0,terraced_house,4,0
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2073,2018-01-03,5.176135e-02,0.998659,2,3,1,0.540641,0.841254,0,0,2.0,terraced_house,4,0
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2073,2018-01-04,6.899114e-02,0.997617,3,4,1,0.540641,0.841254,0,0,2.0,terraced_house,4,0
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2073,2018-01-05,8.620038e-02,0.996278,4,5,1,0.540641,0.841254,0,0,2.0,terraced_house,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185515,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2018-12-27,-5.176135e-02,0.998659,3,27,12,0.540641,0.841254,0,1,3.0,detached_house,1,0
1185516,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2018-12-28,-3.451614e-02,0.999404,4,28,12,0.540641,0.841254,0,1,3.0,detached_house,1,0
1185517,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2018-12-29,-1.726064e-02,0.999851,5,29,12,0.540641,0.841254,1,1,3.0,detached_house,1,0
1185518,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2605,2018-12-30,-2.449294e-16,1.000000,6,30,12,0.540641,0.841254,1,1,3.0,detached_house,1,0


# SKF-CV for training the LGBM
* Using skf on df_train by meter_id with 3 folds
    * Meaning for each iteration we use 2/3 of each meters data for training and 1/3 of each meters data for validating
* using out of fold predictions, making predictions on each fold and aggregating them together for the final prediction

* evaluates the model via OOF predictions made on the held out set on each fold
* this score is returned and along with the 3 models
* grid search will then be used to find the LGBM hyper param config with the best val score and this will then be used to make predictions using the model which has the best validation score


In [20]:
# this lgbm was partly inspired by the work of Wenlong Wu's proposed model : "Solution to the IEEE-CIS Second Technical Challenge with Machine Learning Modeling"

import warnings
warnings.filterwarnings("ignore")

#function for running the cross fold
    #args:
        #disp_fold_info = True if we want to print info for each fold
        #disp_end_info = True if we want to display evaluation info at the end
        #SEED = random seed used for fair repeatability
        #num_folds = number of folds in skf (pretty certain going to keep this at 3 )
        #df_train = training dataframe
        #y_col = the name of the label we want to predict (meter_reading)
        #X_cols = the name of the feature columns we are using
        #X_cat = the name of these features which are categorical
        #params = hyper params for the LGBM model
    #returns:
        #time_of_execution = how long it took to train the model on all folds; will be used as a point of comparison
        #valid_score = MAE calculated using the Out-of-Fold Predictions on the df_train, used for hyper-param tuning
        #lgbm_models = array of the different lgbm_models
        
def run_lgbm_skf_cv(disp_fold_info, disp_end_info, SEED, num_folds, df_train, y_col, X_cols, X_cat, params):
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = SEED) # defining the SKF algorithm

    lgbm_models = []
    start_time = time.time()
    fold_iter=1
    #running the startified kfold, splitting df_train by meter_id, so we use 2/3 of each meters reading for training
    for train_index, valid_index in skf.split(df_train, df_train["meter_id"]):

        print(f"{color.BOLD}{color.BLUE}{color.UNDERLINE}Fold {fold_iter}{color.END}")
        
        #splitting into the features and labels for the train and valid folds
        X_train, X_valid = df_train.loc[train_index, X_cols], df_train.loc[valid_index, X_cols]
        y_train, y_valid = df_train.loc[train_index, y_col], df_train.loc[valid_index, y_col]
        
        if(disp_fold_info):
            print(f"{color.CYAN}{color.UNDERLINE}X_train{color.END}")
            display(X_train.head(5))
            print(f"{color.CYAN}{color.UNDERLINE}y_train{color.END}")
            display(y_train.head(5))
            print(f"{color.CYAN}{color.UNDERLINE}X_valid{color.END}")
            display(X_valid.head(5))
            print(f"{color.CYAN}{color.UNDERLINE}y_valid{color.END}")
            display(y_valid.head(5))
            
        print(f"{color.CYAN}{color.UNDERLINE}Training the LGBM{color.END}")
        #instantiating a lgbm regressor with our params
        lgbm_model = lgbm.LGBMRegressor(**params)
        #fitting the lgbm model on the 2/3 train and evaluating on the 1/3 valid
        #printing details every 1000 iters + stopping if no improvement made in 250 iters
        lgbm_model.fit(X_train, y_train,
                       eval_set=[(X_valid, y_valid)],
                       categorical_feature=X_cat,
                       verbose=1000,
                       early_stopping_rounds=250)
        
        #saving the OOF prediction for the held out rows (valid rows from df_train) from the lgbm model with the best performing intrinisic parmams 
        oof_valid = lgbm_model.predict(X_valid, num_iteration=lgbm_model.best_iteration_) # making prediction on the held out rows, X_valid
        df_train.loc[valid_index, "oof"] = oof_valid #storing the oof rows 
        if(disp_fold_info):
            print(f"{color.CYAN}{color.UNDERLINE}df_train OOF predictions{color.END}")
            display(df_train.loc[valid_index, ["meter_id","oof"]].head(5))
               
        #appending this lgbm
        lgbm_models.append(lgbm_model)
        if(disp_fold_info):
            print(f"{color.CYAN}{color.UNDERLINE}lgbm_models size{color.END}: {len(lgbm_models)}")
            
        fold_iter+=1
    
    #calculating execution time and the MAE on the training set
    time_of_execution = time.time() - start_time
    valid_score=mean_absolute_error(df_train[y_col], df_train["oof"]) 
    
    if(disp_end_info):
        print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}training set OOF preds vs true{color.END}")
        display(df_train[["meter_id","date","meter_reading","oof"]])
        print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}Time of execution for skf-cv:  {color.END}{str(datetime.timedelta(seconds=round(time_of_execution)))}\n")
        print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}Mean absolute error between OOF preds and meter_reading on training set:{color.END} {valid_score}\n")
    
    
    return (time_of_execution, valid_score, lgbm_models)


# Running Grid search on the LGBM skf-cv function
* going to run grid search to find the hyper param config which gav the best MAE on OOF predictions on the train ste in skf-cv
    * exhaustively checking all possible configurations of the possible hyper parameters I specified
        * only doing 1 or 2 of each due to exponential growth of the search space making any more impractical

In [21]:
y_col = "meter_reading" #we want to predict the meter_reading (this will always be the same)

## Defining the arrays of different configs we will use
### Defining the different feature combinations we will use in the grid search
* Multiple configurations to test during hyper parameter tuning
* Only doing 2 configs: one with addInfo and one without as any more will make it infeasible with the exponential growth of the search space

In [37]:
#names of the features we will use
X_cols = [
    #consumption and addInfo
    ["meter_id_ord", "day_of_year_sin", "day_of_year_cos", "day_of_week", "day_of_month", "month_ord",
     "month_sin", "month_cos", "is_weekend", "num_bedrooms", "dwelling_type"],
    
    #only consumption
    ["meter_id_ord","day_of_year_sin","day_of_year_cos","day_of_week","day_of_month",
     "month_ord","month_sin","month_cos","is_weekend"],
    
]

all_cat = ["meter_id_ord", "day_of_week", "day_of_month", "month_ord", "is_weekend", "dwelling_type"] #all categorical values

# setting categorical columns in the dataframe to be categorical
#X_cat[0] holds all columns (besides clusters but we aren't using that) 
for i in all_cat:
    df_train[i] = df_train[i].astype('category')
    df_preds[i] = df_preds[i].astype('category')

### Defining the model parameters for grid search
* sensible different combinations of params for the model

* default values for variables which makes sense (e.g. num_iterations)
* the others which will be tuned as grid search have placeholders to mark them as such

In [56]:
#default params (including ones we will override and those we won't)
params = {
    ### won't be tuned ###
    'boosting_type': 'gbdt', #gbdt/rf/dart/goss
    'metric': 'mae', 
    'num_threads': -1, # number of threads to run on for speed (auto
    'num_iterations': 10000, #defining the models runs
    'seed': SEED, # all runs with same seed for better comparison between different hyper params
    
    
    ### will be tuned (replace these with the best performing) ###
    'learning_rate': 0.99999,
    'num_leaves': 99999, # limit max numer of leaves in a tree
    "max_depth":99999, # limit max depth of the tree to prevent overfitting
    # fraction to be bagged/sampled every k iterations
    'bagging_fraction': 0.99999,
    'bagging_freq' : 99999,
    'feature_fraction': 0.99999, # fraction of features to use at each tree node
    #l1 & l2 regularization to prevent overfitting
    "lambda_l1": 99999,
    "lambda_l2": 99999
}

pprint(params)

{'bagging_fraction': 0.99999,
 'bagging_freq': 99999,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.99999,
 'lambda_l1': 99999,
 'lambda_l2': 99999,
 'learning_rate': 0.99999,
 'max_depth': 99999,
 'metric': 'mae',
 'num_iterations': 10,
 'num_leaves': 99999,
 'num_threads': -1,
 'seed': 137}


In [53]:
learning_rate  =  [0.02]#,0.01]
num_leaves  =  [128]#, 256]
max_depth  =  [12]#,12]
bagging_fraction  =  [0.75,0.9]
bagging_freq  =  [15]
feature_fraction  =  [0.75,0.9]
lambda_l1  =  [6]
lambda_l2  =  [12]

## Running the grid search
* grid search is now outdated only here for the sake of showing my thought process and progress and research

In [58]:
import itertools

num_folds = 3 # 3 fold skf; so we use 2/3 of each meters readings for training at each iteration 
i=0 #just used to print the index of each skf-cv run
all_results = [] # will hold array of all the tuples of results + hyper params

#grid search
print(f"{color.BOLD}Running grid search hyper parameter optimization{color.END}\n")
start_time = time.time()

n=0
#iteratively checking all possible configurations of the specified hyper paramaeters
number_of_combinations = len(X_cols)*len(learning_rate)*len(num_leaves)*len(max_depth)*len(bagging_fraction)*len(bagging_freq)*len(feature_fraction)*len(lambda_l1)*len(lambda_l2)
for grid_params in tqdm(itertools.product(list(range(len(X_cols))), learning_rate,num_leaves,max_depth,bagging_fraction,bagging_freq,feature_fraction,lambda_l1,lambda_l2),total=number_of_combinations):

    #deconstructing the grid_params iterator
    x_index, lr, nl, md, bfrac, bfreq, ff, l1, l2 = grid_params

    #getting the categorical values for this X
    this_X_cats = list(set(X_cols[x_index]).intersection(all_cat)) 
    
    #cloning params and storing this configuration in it
    this_params = params.copy()
    this_params["learning_rate"] = lr
    this_params["max_depth"] = md
    this_params["num_leaves"] = nl
    this_params["bagging_fraction"] = bfrac
    this_params["bagging_freq"] = bfreq
    this_params["feature_fraction"] = ff
    this_params["lambda_l1"] = l1
    this_params["lambda_l2"] = l2

    #train the model with this hyper param config and store it's results
    print(f"{color.BOLD}{color.RED}{color.UNDERLINE}skf-cv run {n}{color.END}")
    print(f"X selection index: {x_index}")
    pprint(this_params)
    all_results.append((run_lgbm_skf_cv(False, False, SEED, num_folds, df_train.copy(), y_col, X_cols[x_index], this_X_cats, this_params),(x_index,this_params)))
    print("\n\n\n")

    n+=1
                                    
time_of_execution = time.time()-start_time
print(f"{color.BOLD}Grid search ran {n} iterations in {str(datetime.timedelta(seconds=round(time_of_execution)))}{color.END}\n") 

Running grid search hyper parameter optimization



  0%|          | 0/8 [00:00<?, ?it/s]

skf-cv run 0
X selection index: 0
{'bagging_fraction': 0.75,
 'bagging_freq': 15,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.75,
 'lambda_l1': 6,
 'lambda_l2': 12,
 'learning_rate': 0.02,
 'max_depth': 12,
 'metric': 'mae',
 'num_iterations': 10,
 'num_leaves': 128,
 'num_threads': -1,
 'seed': 137}
Fold 1
Training the LGBM
[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75
[LightGBM] [Warning] lambda_l1 is set=6, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] lambda_l2 is set=12, reg_lambda=0.0 will be ignored. Current value: lambda_l2=12
[LightGBM] [Warning] bagging_freq is set=15, subsample_freq=0 will be ignored. Current value: bagging_freq=15
Training un

Training until validation scores don't improve for 250 rounds
Did not meet early stopping. Best iteration is:
[10]	valid_0's l1: 4.72505




skf-cv run 3
X selection index: 0
{'bagging_fraction': 0.9,
 'bagging_freq': 15,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.9,
 'lambda_l1': 6,
 'lambda_l2': 12,
 'learning_rate': 0.02,
 'max_depth': 12,
 'metric': 'mae',
 'num_iterations': 10,
 'num_leaves': 128,
 'num_threads': -1,
 'seed': 137}
Fold 1
Training the LGBM
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] lambda_l1 is set=6, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] lambda_l2 is set=12, reg_lambda=0.0 will be ignored. Current value:

Training until validation scores don't improve for 250 rounds
Did not meet early stopping. Best iteration is:
[10]	valid_0's l1: 4.72888




skf-cv run 6
X selection index: 1
{'bagging_fraction': 0.9,
 'bagging_freq': 15,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.75,
 'lambda_l1': 6,
 'lambda_l2': 12,
 'learning_rate': 0.02,
 'max_depth': 12,
 'metric': 'mae',
 'num_iterations': 10,
 'num_leaves': 128,
 'num_threads': -1,
 'seed': 137}
Fold 1
Training the LGBM
[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75
[LightGBM] [Warning] lambda_l1 is set=6, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] lambda_l2 is set=12, reg_lambda=0.0 will be ignored. Current val

In [ ]:
#OLD using nested for loops instead of itertools.product

# num_folds = 3 # 3 fold skf; so we use 2/3 of each meters readings for training at each iteration 
# i=0 #just used to print the index of each skf-cv run
# all_results = [] # will hold array of all the tuples of results + hyper params

# #grid search
# print(f"{color.BOLD}Running grid search hyper parameter optimization{color.END}\n")
# start_time = time.time()

# n=0
# #iteratively checking all possible configurations of the specified hyper paramaeters
# for x_index in range(len(X_cols)):
#     this_X_cats = list(set(X_cols[x_index]).intersection(all_cat)) #getting the categorical values for this X
#     for lr in learning_rate:
#         for nl in num_leaves:
#             for md in max_depth:
#                 for bfrac in bagging_fraction:
#                     for bfreq in bagging_freq:
#                         for ff in feature_fraction:
#                             for l1 in lambda_l1:
#                                 for l2 in lambda_l2:
#                                     #cloning params and storing this configuration in it
#                                     this_params = params.copy()
#                                     this_params["learning_rate"] = lr
#                                     this_params["max_depth"] = md
#                                     this_params["num_leaves"] = nl
#                                     this_params["bagging_fraction"] = bfrac
#                                     this_params["bagging_freq"] = bfreq
#                                     this_params["feature_fraction"] = ff
#                                     this_params["lambda_l1"] = l1
#                                     this_params["lambda_l2"] = l2
                                    
#                                     #train the model with this hyper param config and store it's results
#                                     print(f"{color.BOLD}{color.RED}{color.UNDERLINE}skf-cv run {n}{color.END}")
#                                     print(f"X selection index: {x_index}")
#                                     pprint(this_params)
#                                     all_results.append((run_lgbm_skf_cv(False, False, SEED, num_folds, df_train.copy(), y_col, X_cols[x_index], this_X_cats, this_params),(x_index,this_params)))
#                                     print("\n\n\n")
                                    
#                                     n+=1
                                    
# time_of_execution = time.time()-start_time
# print(f"{color.BOLD}Grid search ran {n} iterations in {str(datetime.timedelta(seconds=round(time_of_execution)))}{color.END}\n") 

## Comparing range of all_results stats

In [ ]:
#inspecting the min/max/mean of time of execution of the models we tested in grid search
all_toes = []
all_maes = [] 

for i in range(len(all_results)):
    #getting the results for this model
    all_toes.append(all_results[i][0][0])
    all_maes.append(all_results[i][0][1])
    
print()
print(f"{color.BOLD}{color.GREEN}{color.UNDERLINE}min time of execution = {color.END}{str(datetime.timedelta(seconds=round(min(all_toes))))}")
print(f"{color.BOLD}{color.GREEN}{color.UNDERLINE}max time of execution = {color.END}{str(datetime.timedelta(seconds=round(max(all_toes))))}")
print(f"{color.BOLD}{color.GREEN}{color.UNDERLINE}mean time of execution = {color.END}{str(datetime.timedelta(seconds=round(np.mean(all_toes))))}")
print()
print(f"{color.BOLD}{color.GREEN}{color.UNDERLINE}min MAE = {color.END}{min(all_maes)}")
print(f"{color.BOLD}{color.GREEN}{color.UNDERLINE}max MAE= {color.END}{max(all_maes)}")
print(f"{color.BOLD}{color.GREEN}{color.UNDERLINE}mean MAE = {color.END}{np.mean(all_maes)}")

## Inspecting the results of the hyper parameter optimization

In [ ]:
best_index = 0
best_mae = 999999

for i in range(len(all_results)):
    #getting the results for this model
    LGBM_TOE = all_results[i][0][0]
    LGBM_MAE = all_results[i][0][1]
    #displaying the results
    print(f"{color.BOLD}{color.RED}{color.UNDERLINE}skf-cv run {i}{color.END}")
    print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}Time of execution for skf-cv:{color.END} {str(datetime.timedelta(seconds=round(LGBM_TOE)))}")
    print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}Mean absolute error between OOF preds and meter_reading on training set:{color.END} {LGBM_MAE}")
    print("")
    
    #keeping track of the best performing model
    if(LGBM_MAE<best_mae):
        best_mae=LGBM_MAE
        best_index = i
print("best model is at index",best_index,"with a MAE of",best_mae)

## Making predictions on best model

In [ ]:
#making predictions based on the best performing model and displaying it's information
BEST_LGBM_MODELS = all_results[best_index][0][2] #getting the lgbm_models from the best index
BEST_LGBM_FORECASTS = df_preds.copy()
start_time = time.time() 

#calculating the average preds by summing the weighted preds for each folds model
for i in range(len(BEST_LGBM_MODELS)):
    pred_forecasts = BEST_LGBM_MODELS[i].predict(BEST_LGBM_FORECASTS[X_cols[all_results[best_index][1][0]]], num_iteration=BEST_LGBM_MODELS[i].best_iteration_) #predicting the unkown df_preds
    BEST_LGBM_FORECASTS[y_col] += pred_forecasts / num_folds #weighting the predictions forBEST_LGBM_FORECASTShis fold and adding to df_preds y column 
BEST_LGBM_FORECASTS["meter_reading"] = BEST_LGBM_FORECASTS.meter_reading.clip(lower=0) #clip meter_reading so no predictions lower than 0
execution_time = time.time() - start_time

## Inspecting information on best model

In [ ]:
#printing the forecasts and other evaluation info
print(f"{color.BOLD}{color.RED}{color.UNDERLINE}best model came from skf-cv run {best_index}{color.END}")
#displaying the results
print(f"{color.BOLD}{color.BLUE}{color.UNDERLINE}Evaluation and results{color.END}")
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}Time of execution for skf-cv:  {color.END}{str(datetime.timedelta(seconds=round(all_results[best_index][0][0])))}\n")
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}Time of execution for predictions: {color.END}{str(datetime.timedelta(seconds=round(execution_time)))}\n")
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}Total time of execution: {color.END}{str(datetime.timedelta(seconds=round(all_results[best_index][0][0]+execution_time)))}\n")
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}Mean absolute error between OOF preds and meter_reading on training set:{color.END} {all_results[best_index][0][1]}\n")
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}preds set with the next years forecasts for each meter{color.END}")
display(BEST_LGBM_FORECASTS[["meter_id","date","meter_reading"]])
#displaying the hyperparameters
print(f"{color.BOLD}{color.BLUE}{color.UNDERLINE}Hyper-parameters{color.END}")
print(f"\n{color.BOLD}{color.CYAN}{color.UNDERLINE}X_cols:{color.END} (index = {all_results[best_index][1][0]})\n")
pprint(X_cols[all_results[best_index][1][0]])
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}model params:  {color.END}")
pprint(all_results[best_index][1][1])